In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 32*32*3)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0, 0, 0), (1, 1, 1))])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

valset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)


model = Net().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', threshold=0.01, patience=5, factor=0.5)


for epoch in range(100):
    acc_temp = []
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        acc_temp = np.append(acc_temp, np.array(torch.argmax(outputs.cpu(), dim=1) == labels.cpu()))

    print(f'Epoch {epoch + 1}, Training Loss: {running_loss / len(trainloader)}')
    print(f'Training Accuracy: {np.mean(acc_temp)}\n')
    scheduler.step(running_loss / len(trainloader))

    with torch.no_grad():
      acc_temp = []
      running_loss = 0.0
      for i, data in enumerate(valloader, 0):
          inputs, labels = data
          inputs = inputs.cuda()
          labels = labels.cuda()

          outputs = model(inputs)
          loss = criterion(outputs, labels)

          running_loss += loss.item()
          acc_temp = np.append(acc_temp, np.array(torch.argmax(outputs.cpu(), dim=1) == labels.cpu()))

    print(f'Epoch {epoch + 1}, Valdiation Loss: {running_loss / len(trainloader)}')
    print(f'Validation Accuracy: {np.mean(acc_temp)}\n')


Files already downloaded and verified
Files already downloaded and verified
Epoch 1, Training Loss: 1.8484972009573446
Training Accuracy: 0.32674

Epoch 1, Valdiation Loss: 0.3462818194838131
Validation Accuracy: 0.3744

Epoch 2, Training Loss: 1.6638458304088135
Training Accuracy: 0.402

Epoch 2, Valdiation Loss: 0.3259813537073257
Validation Accuracy: 0.4197

Epoch 3, Training Loss: 1.573893328609369
Training Accuracy: 0.43914

Epoch 3, Valdiation Loss: 0.32004432315411774
Validation Accuracy: 0.4298

Epoch 4, Training Loss: 1.5114428960453823
Training Accuracy: 0.46006

Epoch 4, Valdiation Loss: 0.30836309039074444
Validation Accuracy: 0.4505

Epoch 5, Training Loss: 1.4703317728188947
Training Accuracy: 0.47538

Epoch 5, Valdiation Loss: 0.2958682023960611
Validation Accuracy: 0.4723

Epoch 6, Training Loss: 1.4319250350413115
Training Accuracy: 0.48684

Epoch 6, Valdiation Loss: 0.2944205278325874
Validation Accuracy: 0.477

Epoch 7, Training Loss: 1.4056733121042666
Training Accu